In [3]:
from flask import Flask, jsonify, request
import numpy as np
import joblib
import pandas as pd
import numpy as np
from sklearn import linear_model
import joblib
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

Completed!


In [5]:
# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2022-10-09 10:23:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  5.93MB/s    in 2.2s    

2022-10-09 10:23:06 (5.93 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]



In [6]:
# extract the downloaded file using the following command 

!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [7]:
# paste your AuthToken here and execute this command

!./ngrok authtoken 2EM30ZjrtTpWthpgW8huDyOBOPb_uxKo2u6r6VDaiTVQZoyC

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def preproc(sentance):
#from tqdm import tqdm
# tqdm is for printing the status bar
#for sentance in tqdm(df['combined'].values):
    sentance = re.sub(r"http\S+", "url", sentance)  #removing urls with space
    sentance = BeautifulSoup(sentance, 'lxml').get_text() # removes tags like <br>
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()  # remove words with numbers 
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)  ##remove spacial character:
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    
    return(sentance.strip())


In [76]:

def predict_proba(arr):
  print(arr)
  processed=[]
  for i in arr:
    processed.append(preproc(i))
  list_tokenized_ex = tokenizer.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=600)
  pred=clf.predict(Ex)
  
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  
  return np.array(returnable)

In [ ]:
# import Flask from flask module
from flask import Flask
import os

PEOPLE_FOLDER = os.path.join('static', 'people_photo')


# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template , request 

  
app = Flask(__name__) #app name
run_with_ngrok(app)

IMG_FOLDER = os.path.join('static', 'IMG')

app.config['UPLOAD_FOLDER'] = IMG_FOLDER

''' 
@app.route("/")
def hello():
    return "Hello Friends! from Pykit.org. Thank you! for reading this article."
'''
@app.route("/")
def index():
    return render_template("index.html")


@app.route('/predict', methods=['POST'])
def predict():
  
  to_predict = request.form.to_dict()
  #to_predict_text = request.form.to_dict()
  news_title = preproc(to_predict['news_title'])
  news_text = preproc(to_predict['news_text'])
  
  clf = joblib.load('model.pkl')
  tokenizer = joblib.load('tokenizer.pkl')
  combined=news_title + news_text
  processed=[]
  processed.append(preproc(combined))
 
  list_tokenized_ex = tokenizer.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=600)
  pred = clf.predict(Ex)
  if pred[0]>=0.5:
      prediction = "real news"
  else:
      prediction = "Fake News"
  
  
  from lime.lime_text import LimeTextExplainer
  class_names=['0','1']
  explainer= LimeTextExplainer(class_names=class_names)
  import matplotlib.pyplot as plt
  exp =explainer.explain_instance(combined,predict_proba)
  #exp.show_in_notebook(text=True)
  #exp.available_labels()
  exp.as_pyplot_figure(label=1,)
  plt.savefig("/content/static/IMG/images.png")
  #plt.show()



  
  
  Flask_Logo = os.path.join(app.config['UPLOAD_FOLDER'], 'images.png')

  return render_template('/predict.html', prediction_text = prediction ,user_image=Flask_Logo)

    

if __name__ == "__main__":
    app.run()
  

 

  #full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'shovon.png')
  #return render_template("index.html", prediction_text=' this is a {}'.format(prediction),user_image = full_filename)

  #return render_template('index.html', prediction_text=' this is a {}'.format(prediction),user_image ="/content/templates/squares234.png")
  

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6496-35-236-186-229.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [09/Oct/2022 14:10:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Oct/2022 14:10:56] "GET /favicon.ico HTTP/1.1" 404 -


['suryamunjal love chitz', '  ', '  chitz', ' love ', '  ', 'suryamunjal love ', '  ', 'suryamunjal  ', '  ', 'suryamunjal love ', ' love ', ' love ', 'suryamunjal  ', 'suryamunjal love ', 'suryamunjal love ', '  ', ' love ', ' love chitz', 'suryamunjal  chitz', 'suryamunjal  chitz', 'suryamunjal  chitz', '  chitz', ' love ', 'suryamunjal  ', '  ', 'suryamunjal love ', 'suryamunjal  chitz', ' love chitz', '  ', 'suryamunjal love ', '  chitz', '  ', '  ', '  ', '  ', '  chitz', 'suryamunjal love ', '  ', ' love ', '  chitz', '  ', 'suryamunjal love ', 'suryamunjal  chitz', 'suryamunjal  ', ' love ', '  ', 'suryamunjal  chitz', ' love chitz', ' love chitz', 'suryamunjal  ', 'suryamunjal love ', '  ', 'suryamunjal  chitz', 'suryamunjal  ', 'suryamunjal  chitz', '  ', ' love chitz', '  ', ' love chitz', 'suryamunjal  ', '  ', '  ', '  chitz', ' love ', '  chitz', 'suryamunjal  chitz', '  ', '  ', ' love ', '  ', 'suryamunjal love ', 'suryamunjal  chitz', '  chitz', 'suryamunjal  chitz', 's

INFO:werkzeug:127.0.0.1 - - [09/Oct/2022 14:11:29] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Oct/2022 14:11:30] "GET /static/IMG/images.png HTTP/1.1" 200 -


In [11]:
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
